<a href="https://colab.research.google.com/github/crudolphMQU/programmingbitcoin/blob/master/code-ch02/Chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use colab with 'Programming Bitcoin'



## Notes on saving files to GitHub via Colab

Saving to GitHub from Colab **only saves the notebook**, not any additional files.

When you edit the `ecc.py` file in your Colab instance, you should **also update it directly in your GitHub repository**. Editing it in Colab allows you to continue working in the same session, but if you start a new runtime later, those changes will not persist unless they are committed to GitHub.

Simply uploading the `ecc` file to GitHub often doesn't work as expected. Instead, you should **edit the file directly in the appropriate folder of your repository on GitHub**.

After deleting the runtime and restarting the notebook, you'll get the most up-to-date version of `ecc.py` from GitHub. If you don't update it properly, you may encounter errors when running the notebook in a new runtime.

Whenever you make changes to `ecc.py`, reload it using the following code:

```python
import importlib
import ecc
import importlib.reload(ecc)
```

When saving a copy of your notebook to GitHub, make sure to **specify the correct folder path**. For example, for this chapter:

```
File > Save a copy in GitHub > Choose your repository > Filepath: code-ch02/Chapter2.ipynb
```


## ✅ Step-by-step instructions: Saving files to GitHub via Colab

### 1️⃣ Understand what gets saved

- When you use **File → Save a copy in GitHub**, only the notebook file (`.ipynb`) is saved.
- Other files you edit in Colab (like `ecc.py`) **do not get automatically saved to GitHub**.

---

### 2️⃣ Update `ecc.py` manually

- If you edit `ecc.py` in Colab, you **must also update it directly in your GitHub repository**.
- You can do this by:
  - Editing the file directly on GitHub (recommended), or
  - Uploading via the GitHub web interface (make sure to place it in the correct folder).

---

### 3️⃣ Check the correct file path when saving notebooks

- When saving your notebook to GitHub, always specify the correct path.
- Example:


# set up and imports

In [1]:
!git clone https://github.com/crudolphMQU/programmingbitcoin

fatal: destination path 'programmingbitcoin' already exists and is not an empty directory.


In [2]:
ls

programmingbitcoin/  sample_data/


In [3]:
cd programmingbitcoin/code-ch02

/content/programmingbitcoin/code-ch02


In [4]:
!pip install virtualenv==20.17.1

In [5]:
!pip install -r /content/programmingbitcoin/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
  Attempting uninstall: jupyter-server
    Found existing installation: jupyter-server 1.16.0
    Uninstalling jupyter-server-1.16.0:
      Successfully uninstalled jupyter-server-1.16.0


In [6]:
!pip install helper

  Preparing metadata (setup.py) ... done
  Created wheel for helper: filename=helper-2.5.0-py2.py3-none-any.whl size=19169 sha256=28cc087d71cbe248030bcdb8e3a8e7314f7a5434e6590d8d290f5fce41b056ca
  Stored in directory: /root/.cache/pip/wheels/8b/ab/a7/06bcd4f2d89113bb7b0caaf5562aa209a27863d8cccb07a1bc
Successfully built helper


In [7]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
import helper

from helper import run
import ecc

from ecc import Point

Using the curve in fig 2-9

$y^2 = x^3 + 5x + 7$

defined in general form in the `ecc.py` file, we're interested in the following points:

In [8]:
from ecc import Point
p1 = Point(-1, -1, 5, 7)
# p2 = Point(-1, -2, 5, 7)

running p2 will raise the ValueError from the ecc file advising the point is not on the curve. p2 is simply written to show that it is not on the curve, no need to change

# Exercise 1

Determine which of these points are on the curve \\(y^{2}\\)=\\(x^{3}\\)+5x+7:

(2,4), (-1,-1), (18,77), (5,7)

In [9]:
# Exercise 1

# (2,4), (-1,-1), (18,77), (5,7)
# equation in python is: y**2 == x**3 + 5*x + 7

In [10]:
# Curve parameters
a = 5
b = 7

# List of points to check
points = [(2, 4), (-1, -1), (18, 77), (5, 7)]

# Check each point
for x, y in points:
    left = y**2
    right = x**3 + a*x + b
    if left == right:
        print(f"Point ({x}, {y}) is ON the curve.")
    else:
        print(f"Point ({x}, {y}) is NOT on the curve.")


Point (2, 4) is NOT on the curve.
Point (-1, -1) is ON the curve.
Point (18, 77) is ON the curve.
Point (5, 7) is NOT on the curve.


# Exercise 2

Write the `__ne__` method for `Point`.

#### Make [this test](/edit/code-ch02/ecc.py) pass: `ecc.py:PointTest:test_ne`

In [11]:
import importlib
import ecc
importlib.reload(ecc)

<module 'ecc' from '/content/programmingbitcoin/code-ch02/ecc.py'>

In [12]:
ls

answers.py      ecc.py       helper.py    __pycache__/
Chapter2.ipynb  examples.py  jupyter.txt


In [13]:
# Exercise 2

reload(ecc)
run(ecc.PointTest("test_ne"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


The purpose of definiing the method `__ne__` in this way is to avoid writing the entire definition, we're using the same logic definied with `__eq__`.

```python
def __ne__(self, other):
    return not (self == other)
```

Commutative addition: adding point A to B gives the same result as adding point B to point A. Elliptic curves use 'point addition' which adds two points that are on the curves to give a third point on also on the curve.

lines on a curve can intercept once or three times. It's also possible to intercept two times, where one of the two interceptions is tangent to the curve. The other case is when the line is vertical, and therefore must intercet at two points.

Point addition can be not easily predictable.

Point addition satisfies identity, commutativity, associativity, invertibility.

Coding point addition:
1.

`Inf` doesn't work well in python so we use `None`.

We want the following to work

```python

p1 = Point(-1, -1, 5, 7)
p2 = Point(-1, 1, 5, 7)
inf = Point(None, None, 5, 7)
print(p1 + inf)
print(inf + p2)
print(p1 + p2)

```

First change the `__init__` method to prevent checking the curve equation is satisfied when a point at inifity is provided.

Secondly, overload the addition operator (` __add__`).

**are methods contained in classes?(yes, but not sure if methods are limited to being contained in classes), `FieldElement` is a class, is `__init__` a method in the `Point` class? (yes)**

In [55]:
from ecc import Point
p1 = Point(-1, -1, 5, 7)
p2 = Point(-1, 1, 5, 7)
inf = Point(None, None, 5, 7)

print(p1 + inf)

Point(-1,-1)_5_7


Above output is p1 (itself). Adding inf to any point returns returns itself.

This uses the *Identity* property of *Point Addition*:

*I + A = A*

Where *A* is point *A* and *I* is a point at inf.  

In [16]:
print(inf + p2)

Point(-1,1)_5_7


Testing the *Identity* property again with p2 is correct.

To get the following point addition to work we have to understand what is going on. p1 and p2 have the same x coordinate but opposite y coordinate. This results in the **Case 1** (a point at infinity)

In [56]:
import importlib
import ecc
importlib.reload(ecc)

from ecc import Point

<module 'ecc' from '/content/programmingbitcoin/code-ch02/ecc.py'>

In [57]:
print(p1 + p2)

Point(infinity)


It is important to import the updated `Point` class again

# Exercise 3


Handle the case where the two points are additive inverses. That is, they have the same `x`, but a different `y`, causing a vertical line. This should return the point at infinity.

#### Make [this test](/edit/code-ch02/ecc.py) pass: `ecc.py:PointTest:test_add0`

This was done in the previous exercise when we added p1 and p2, the x coordinate is the same, but the y coordinate is the inverse, which creates a vertical line.

In [58]:
# Exercise 3

reload(ecc)
run(ecc.PointTest("test_add0"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


# Exercise 4


For the curve \\(y^{2}\\)=\\(x^{3}\\)+5x+7, what is (2,5) + (-1,-1)?

In [ ]:
# Exercise 4

from ecc import Point

a = 5
b = 7
x1, y1 = 2, 5
x2, y2 = -1, -1

# (x1,y1) + (x2,y2)

# Exercise 5

Write the `__add__` method where \\(x_{1}\\)≠\\(x_{2}\\)

#### Make [this test](/edit/code-ch02/ecc.py) pass: `ecc.py:PointTest:test_add1`

In [ ]:
# Exercise 5

reload(ecc)
run(ecc.PointTest("test_add1"))

# Exercise 6



For the curve \\(y^{2}\\)=\\(x^{3}\\)+5x+7, what is (-1,-1) + (-1,-1)?

In [ ]:
# Exercise 6

from ecc import Point

a = 5
b = 7
x1, y1 = -1, -1
# (-1,-1) + (-1,-1)

# Exercise 7

Write the `__add__` method when \\(P_{1}\\)=\\(P_{2}\\).

**Make [this test](/edit/code-ch02/ecc.py) pass: `ecc.py:PointTest:test_add2`**

In [ ]:
# Exercise 7

reload(ecc)
run(ecc.PointTest("test_add2"))